# Q3 

Language Modeling (30 points)

Implement a 4-gram language model from the following English training data: http://www.nltk.org/nltk_data/packages/corpora/gutenberg.zip
Across all files in the directory (counted together), report the 4-gram counts.
Refer instructions below.



For the given datasets:
News Articles:
https://drive.google.com/open?id=14veC-71fA_pE-kR8GUz8xwV37NehPrA2

IMDB Movie Reviews:
https://drive.google.com/open?id=1AaErcFL4H30LVdiObQoY4w7MIVzPCVXq


In [1]:
import os
import time
import re
import pickle
import math
from collections import Counter

In [2]:
## get data and preprocess it
def data_extractor(dataset):
    complete_data = []
    filename = os.getcwd() + "\\" + dataset
    for text in os.listdir(filename):
        filename = filename + "\\" + text
        if ".DS_Store" not in filename:
            with open(filename) as file:
                text_in_file = file.read()
                text_in_file = text_in_file.lower() ## make all text lowercase
                text_in_file = text_in_file.replace("[", "[ ").replace("]", " ]").replace(")", ") ").replace("(", " (").replace("}", " }").replace("{", "{ ")
                text_in_file = re.sub("\n+", " ", text_in_file) ## remove extra lines
                text_in_file = re.sub("\s+", " ", text_in_file) ## remove extra spaces
            text_in_file += " "
            complete_data.append(text_in_file)
        filename = os.getcwd() + "\\" + dataset
    return complete_data
    

## 2.1
Calculate the average perplexity for each dataset. Use lambda smoothing (with lambda =
0.1)


For preprocessing the data, see instructions below.
Data preprocessing instructions for 3.1 and 3.2:
1. Remove blank lines from each file.
2. Replace newline characters with spaces.
3. Remove duplicate spaces.
4. Tokenize each document (you can use NLTK, https://www.nltk.org/).
5. Replace words in train set that appear less than 5 times as \UNK".

In [3]:
## get data in as a single string then split to get each token, calculate unigram count
text_in_file = data_extractor(dataset = "gutenberg")
complete_data = " ".join(text_in_file)
complete_data_list = complete_data.split(" ")
len_set_complete_data = len(list(set(complete_data_list)))
data_unigram_count = Counter(complete_data_list)
data_count_less_than_5 = {i: data_unigram_count[i] for i in data_unigram_count if data_unigram_count[i] < 5}

In [18]:
## change the less frequent words to "UNK" token
for index, i in enumerate(data_count_less_than_5):
    word = " " + i + " "
    complete_data = complete_data.replace(word , " UNK ")
    if index % 1000 == 0:
        print (index)
    

In [5]:
## get tri grams and their count
complete_data_list = complete_data.split(" ")
all_3_grams = [" ".join(complete_data_list[index:index + 3]) for index in range(len(complete_data_list[:-2]))]
count_3_grams = Counter(all_3_grams)

In [22]:
count_3_grams

Counter({'[ emma by': 1,
         'emma by jane': 1,
         'by jane UNK': 3,
         'jane UNK UNK': 7,
         'UNK UNK ]': 23,
         'UNK ] volume': 1,
         '] volume i': 1,
         'volume i chapter': 1,
         'i chapter i': 1,
         'chapter i emma': 2,
         'i emma woodhouse,': 1,
         'emma woodhouse, handsome,': 1,
         'woodhouse, handsome, clever,': 1,
         'handsome, clever, and': 1,
         'clever, and rich,': 1,
         'and rich, with': 1,
         'rich, with a': 1,
         'with a comfortable': 1,
         'a comfortable home': 2,
         'comfortable home and': 1,
         'home and happy': 1,
         'and happy disposition,': 1,
         'happy disposition, seemed': 1,
         'disposition, seemed to': 1,
         'seemed to unite': 1,
         'to unite some': 1,
         'unite some of': 1,
         'some of the': 112,
         'of the best': 23,
         'the best blessings': 2,
         'best blessings of': 2,
         'ble

In [7]:
## get four grams and their count
all_4_grams = [" ".join(complete_data_list[index:index + 4]) for index in range(len(complete_data_list[:-3]))]
count_4_grams = Counter(all_4_grams)

In [8]:
count_4_grams

Counter({'[ emma by jane': 1,
         'emma by jane UNK': 1,
         'by jane UNK UNK': 3,
         'jane UNK UNK ]': 3,
         'UNK UNK ] volume': 1,
         'UNK ] volume i': 1,
         '] volume i chapter': 1,
         'volume i chapter i': 1,
         'i chapter i emma': 1,
         'chapter i emma woodhouse,': 1,
         'i emma woodhouse, handsome,': 1,
         'emma woodhouse, handsome, clever,': 1,
         'woodhouse, handsome, clever, and': 1,
         'handsome, clever, and rich,': 1,
         'clever, and rich, with': 1,
         'and rich, with a': 1,
         'rich, with a comfortable': 1,
         'with a comfortable home': 1,
         'a comfortable home and': 1,
         'comfortable home and happy': 1,
         'home and happy disposition,': 1,
         'and happy disposition, seemed': 1,
         'happy disposition, seemed to': 1,
         'disposition, seemed to unite': 1,
         'seemed to unite some': 1,
         'to unite some of': 1,
         'unite so

## 2.2
For the given datasets:
News Articles:

https://drive.google.com/open?id=14veC-71fA_pE-kR8GUz8xwV37NehPrA2
IMDB Movie Reviews:

https://drive.google.com/open?id=1AaErcFL4H30LVdiObQoY4w7MIVzPCVXq

Calculate the average perplexity for each dataset. Use lambda smoothing (with lambda =
0.1)
For preprocessing the data, see instructions below.

In [11]:
## find perplexity based on training count
def calculate_perplexity(test_complete_dataset):
    all_perplexity = []
    for test_set in test_complete_dataset[:1]:
        test_set_list = test_set.split(" ")
        not_preset_in_training_words = set(test_set_list) - set(complete_data_list) 
        for i in not_preset_in_training_words:
            word = " " + i + " "
            test_set = test_set.replace(word , " UNK ")  
        test_set_list = test_set.split(" ")
        test_4_grams = [" ".join(test_set_list[index:index + 4]) for index in range(len(test_set_list[:-3]))]
        lambda_val = 0.1
        test_prob_4_gram = {}
        vocab = len(set(test_set_list))
        for four_gram in test_4_grams:
            word_in_three_gram = i.split(" ")[:-1]
            three_gram = " ".join(word_in_three_gram)
            if four_gram not in count_4_grams:
                count_4_grams[four_gram] = 0
            if three_gram not in count_3_grams:
                count_3_grams[three_gram] = 0
            test_prob_4_gram[four_gram] = (count_4_grams[four_gram] + lambda_val)/ (count_3_grams[three_gram] + lambda_val * vocab)
        entropy = 0
        for i in test_prob_4_gram:
            entropy += math.log(test_prob_4_gram[i])
        perplexity_per_doc = (math.e ** (-1/(len(test_set_list)) * entropy))
        all_perplexity.append(perplexity_per_doc)
    avg_perplexity = sum(all_perplexity)/len(all_perplexity)
    return avg_perplexity

In [15]:
## call dataset gatherer and perplexity calculator
def get_avg_perlexity(dataset):
    test_complete_dataset = data_extractor(dataset)
    avg_perplexity = calculate_perplexity(test_complete_dataset)
    print ("Average perplexity for {0} dataset: {1}".format(dataset, avg_perplexity))

In [16]:
get_avg_perlexity(dataset = "imdb_data")

Average perplexity for imdb_data dataset: 70.17773402921019


In [17]:
get_avg_perlexity(dataset = "news_data")

Average perplexity for news_data dataset: 128.4372991933948
